In [ ]:
import numpy as np
import scipy as sp
from numpy import linalg as la
import matplotlib.pyplot as plt
from numpy import linalg as LA;
from scipy import linalg as LA2;
from numpy import random as rand
from scipy.sparse import diags
from scipy.sparse import csr_matrix

In [ ]:
def tensorvect(a,b):
    return(np.tensordot(a,b,axes=0).flatten())

def tensorvectop(a,b):
    return np.kron(a,b)

def opchain(a,i,nspin):
    if i==1:
        return np.kron(a,np.identity(2**(nspin-1)))
    else:
        if i==nspin:
            return np.kron(np.identity(2**(nspin-1)),a)
        else:
            return np.kron(np.kron(np.identity(2**(i-1)),a),np.identity(2**(nspin-i)))
        
def opchain2(a,i,b,j,nspin):
    if i==1:
        if j==nspin:
            return np.kron(np.kron(a,np.identity(2**(nspin-2))),b)
        else:
            return np.kron(np.kron(a,np.identity(2**(j-2))),np.kron(b,np.identity(2**(nspin-j))))      
    else:
        if j==nspin:
            return np.kron(np.kron(np.identity(2**(i-1)),a),np.kron(np.identity(2**(nspin-(i+1))),b))
        else:
            return np.kron(np.kron(np.kron(np.identity(2**(i-1)),a),np.kron(np.identity(2**(j-(i+1))),b)),np.identity(2**(nspin-j)))
            
def buildstate(bin):
    v=[0. for i in range(2**len(bin))];
    v[int(bin,2)]=1.
    return np.array(v)


def diracrep(psi,nspin):
    state='';
    for i in range(2**nspin):
        if abs(psi[i])>10**(-6):
            state=state+'+'+str(psi[i])+'|'+format(i,'0'+str(nspin)+'b')+'>'
    return state

def binnum(n):
    l=['0','1'];
    if n==1:
        return l
    else:
        return ['0'+i for i in binnum(n-1)]+['1'+i for i in binnum(n-1)]
    
def densmat(psi,i,nspin):
    if i>1:
        listindex0=binnum(i-1)
        listindex0=[j+'0' for j in listindex0]
    else:
        listindex0=['0']
    if i<nspin:
        listcomp=binnum(nspin-i)
        listindex0=list(np.array([[j+k for k in listcomp] for j in listindex0]).flatten())
    if i>1:
        listindex1=binnum(i-1)
        listindex1=[j+'1' for j in listindex1]
    else:
        listindex1=['1']
    if i<nspin:
        listcomp=binnum(nspin-i)
        listindex1=list(np.array([[j+k for k in listcomp] for j in listindex1]).flatten())
    rho00=sum(psi[int(j,2)]*np.conjugate(psi[int(j,2)]) for j in listindex0)
    rho11=sum(psi[int(j,2)]*np.conjugate(psi[int(j,2)]) for j in listindex1)
    rho01=sum(psi[int(j,2)]*np.conjugate(psi[int(listindex1[listindex0.index(j)],2)]) for j in listindex0)
    return np.array([[rho00,rho01],[np.conjugate(rho01),rho11]])

def avdensmat(psi,nspin):
    rho=densmat(psi,1,nspin);
    if nspin>1:
        for i in range(2,nspin+1):
            rho=rho+densmat(psi,i,nspin)
    rho=rho/nspin;
    return rho

def purity(rho):
    rho2 = np.dot(rho,rho)
    tr = np.trace(rho2)
    return(tr)

def SvN(rho):
    vp=np.real(LA.eigvals(rho));
    S=0.;
    for i in range(len(vp)):
        if vp[i]>0.:
            S=S+vp[i]*np.log(vp[i])
    return -S

def entangl(psi,nspin):
    S=SvN(densmat(psi,1,nspin));
    if nspin>1:
        for i in range(2,nspin+1):
            S=S+SvN(densmat(psi,i,nspin))
    return S/nspin

def Disorder(psi,nspin):
    return SvN(avdensmat(psi,nspin))-entangl(psi,nspin)

In [ ]:
sigX=np.array([[0.,1.],[1.,0.]]);
sigY=np.array([[0.,-1j],[1j,0.]]);
sigZ=np.array([[1.,0.],[0.,-1.]]);
sig1=np.array([[1.,0.],[0.,0.]]);
id2 =np.array([[1.,0.],[0.,1.]]);

NOT=sigX;
HAD1=np.array([[1./np.sqrt(2.),1./np.sqrt(2.)],[1./np.sqrt(2.),-1./np.sqrt(2.)]]);
CNOT=np.array([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,0.,1.],[0.,0.,1.,0.]]);
HAD2=np.array([[0.5,0.5,0.5,0.5],[0.5,-0.5,0.5,-0.5],[0.5,0.5,-0.5,-0.5],[0.5,-0.5,-0.5,0.5]]);
SWAP=np.array([[1.,0.,0.,0.],[0.,0.,1.,0.],[0.,1.,0.,0.],[0.,0.,0.,1.]]);

# Student code starts here:

In [ ]:
def QuantumHamiltonian_chain(Jxyz, w, N, flag, force_real):
    h0 = np.array([[-w,0],[0,0]])
    H0 = np.zeros(shape=(2**N,2**N), dtype=np.complex128)
    Hint = np.copy(H0)
    for i in range(1,N+1):
        H0 += opchain(h0,i,N)
        
    for i in range(1,N):
        j = i+1

        Hint -= (Jxyz[0]*opchain2(0.5*sigX,i,0.5*sigX,j,N)+Jxyz[1]*opchain2(0.5*sigY,i,0.5*sigY,j,N)+Jxyz[2]*opchain2(0.5*sigZ,i,0.5*sigZ,j,N))
    if (flag=="close" and N>2):
        Hint -= (Jxyz[0]*opchain2(0.5*sigX,1,0.5*sigX,N,N)+Jxyz[1]*opchain2(0.5*sigY,1,0.5*sigY,N,N)+Jxyz[2]*opchain2(0.5*sigZ,1,0.5*sigZ,N,N))
    H = H0+Hint

    if force_real == True:
        H = H.real

    return H

def QuantumHamiltonian_general(J, w, N, flag, force_real):
    if type(w)==float:
        h0 = np.array([[-w,0],[0,0]])
    H0 = np.zeros(shape=(2**N,2**N), dtype=np.complex128)
    Hint = np.copy(H0)
    for i in range(1,N+1):
        if type(w)!=float:
            h0 = np.array([[-w[i-1],0],[0,0]])
        H0 += opchain(h0,i,N)
        
    for i in range(1,N+1):
        for j in range(1,N+1):
            if j>i:
                Hint -= (J[i-1,j-1,0]*opchain2(0.5*sigX,i,0.5*sigX,j,N)+J[i-1,j-1,1]*opchain2(0.5*sigY,i,0.5*sigY,j,N)+J[i-1,j-1,2]*opchain2(0.5*sigZ,i,0.5*sigZ,j,N))
    if (flag=="close" and N>2):
        Hint -= (J[0,N-1,0]*opchain2(0.5*sigX,1,0.5*sigX,N,N)+J[0,N-1,1]*opchain2(0.5*sigY,1,0.5*sigY,N,N)+J[0,N-1,2]*opchain2(0.5*sigZ,1,0.5*sigZ,N,N))
    H = H0+Hint

    if force_real == True:
        H = H.real

    return H

### 2.1 Studied models: Hamiltonian tests for different lattices and different sizes (N=2,3,8):

In [ ]:
# (1) open Ising-Z spin chain

H_oIZ_2 = QuantumHamiltonian_chain(Jxyz=[0,0,1], w=0.5, N=2, flag="open", force_real=True)
H_oIZ_3 = QuantumHamiltonian_chain(Jxyz=[0,0,1], w=0.5, N=3, flag="open", force_real=True)
H_oIZ_8 = QuantumHamiltonian_chain(Jxyz=[0,0,1], w=0.5, N=8, flag="open", force_real=True)

print(csr_matrix(H_oIZ_2))
print(csr_matrix(H_oIZ_3))
print(csr_matrix(H_oIZ_8))

In [ ]:
# (2) closed Ising-Z spin chain

H_cIZ_2 = QuantumHamiltonian_chain(Jxyz=[0,0,1], w=0.5, N=2, flag="close", force_real=True)
H_cIZ_3 = QuantumHamiltonian_chain(Jxyz=[0,0,1], w=0.5, N=3, flag="close", force_real=True)
H_cIZ_8 = QuantumHamiltonian_chain(Jxyz=[0,0,1], w=0.5, N=8, flag="close", force_real=True)

print(csr_matrix(H_cIZ_2))
print(csr_matrix(H_cIZ_3))
print(csr_matrix(H_cIZ_8))

In [ ]:
# (3) open Ising-X spin chain

H_oIX_2 = QuantumHamiltonian_chain(Jxyz=[1,0,0], w=0.5, N=2, flag="open", force_real=True)
H_oIX_3 = QuantumHamiltonian_chain(Jxyz=[1,0,0], w=0.5, N=3, flag="open", force_real=True)
H_oIX_8 = QuantumHamiltonian_chain(Jxyz=[1,0,0], w=0.5, N=8, flag="open", force_real=True)

print(csr_matrix(H_oIX_2))
print(csr_matrix(H_oIX_3))
print(csr_matrix(H_oIX_8))

In [ ]:
# (4) open Heisenberg-XXX spin chain

H_oHXXX_2 = QuantumHamiltonian_chain(Jxyz=[1,1,1], w=0.5, N=2, flag="open", force_real=True)
H_oHXXX_3 = QuantumHamiltonian_chain(Jxyz=[1,1,1], w=0.5, N=3, flag="open", force_real=True)
H_oHXXX_8 = QuantumHamiltonian_chain(Jxyz=[1,1,1], w=0.5, N=8, flag="open", force_real=True)

print(csr_matrix(H_oHXXX_2))
print(csr_matrix(H_oHXXX_3))
print(csr_matrix(H_oHXXX_8))

In [ ]:
# (5) open Heisenberg-XYZ spin chain

H_oHXXX_2 = QuantumHamiltonian_chain(Jxyz=[0.5,1,1.5], w=0.5, N=2, flag="open", force_real=True)
H_oHXXX_3 = QuantumHamiltonian_chain(Jxyz=[0.5,1,1.5], w=0.5, N=3, flag="open", force_real=True)
H_oHXXX_8 = QuantumHamiltonian_chain(Jxyz=[0.5,1,1.5], w=0.5, N=8, flag="open", force_real=True)

print(csr_matrix(H_oHXXX_2))
print(csr_matrix(H_oHXXX_3))
print(csr_matrix(H_oHXXX_8))

In [ ]:
# (6) open random Ising-X spin chain with random Larmor frequencies

np.random.seed(0)

w_rand = lambda N: np.random.uniform(0,0.5,N)
def build_J_X(N):
    J = np.zeros(shape=(N,N,3))
    for i in range(N):
        for j in range(N):
            if j>i:
                J[i,j,0] = np.random.uniform(-1,1)
    return J

H_orIX_2 = QuantumHamiltonian_general(build_J_X(2), w_rand(2), N=2, flag="open", force_real=True)
H_orIX_3 = QuantumHamiltonian_general(build_J_X(3), w_rand(3), N=3, flag="open", force_real=True)
H_orIX_8 = QuantumHamiltonian_general(build_J_X(8), w_rand(8), N=8, flag="open", force_real=True)

print(f"N=2:\n{H_orIX_2}\n\n")
print(f"N=3 (we rounded random outputs to see a proper matrix on screen):\n{np.around(H_orIX_3,3)}\n\n")

print(csr_matrix(H_orIX_2))
print(csr_matrix(H_orIX_3))
print(csr_matrix(H_orIX_8))



In [ ]:
# (7) open random Spin Glass with random Larmor frequencies

np.random.seed(0)

w_rand = lambda N: np.random.uniform(0,0.5,N)
def build_JXYZ(N):
    J = np.zeros(shape=(N,N,3))
    for i in range(N):
        for j in range(N):
            if j>i:
                J[i,j] = np.random.uniform(-1,1,3)
    return J

H_SP_2 = QuantumHamiltonian_general(build_JXYZ(2), w_rand(2), N=2, flag="close", force_real=True)
H_SP_3 = QuantumHamiltonian_general(build_JXYZ(3), w_rand(3), N=3, flag="close", force_real=True)
H_SP_8 = QuantumHamiltonian_general(build_JXYZ(8), w_rand(8), N=8, flag="close", force_real=True)

print(f"N=2:\n{H_SP_2}\n\n")


print(f"N=3 (we rounded random outputs to see a proper matrix on screen):\n{np.around(H_SP_3,3)}\n\n")

print(csr_matrix(H_SP_2))
print(csr_matrix(H_SP_3))
print(csr_matrix(H_SP_8))

# 2.2 Diagonalization algorithms - Power Method algorithm 

In [ ]:
def pm_0(mu, eps=1e-8, kmax=int(1e3)):
    